In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `first_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `first_token`

In [3]:
import pandas as pd

conc_dataset_path = '/content/drive/My Drive/CS 5787/Project/OpenCHAIR-Adjectives/Concreteness_ratings_Brysbaert_et_al_BRM.xlsx'
word_conc = pd.read_excel(conc_dataset_path)[['Word','Conc.M']].set_index("Word").to_dict()['Conc.M']

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from datasets import load_dataset
print("Loading Dataset\n")
och_dataset = load_dataset("moranyanuka/OpenCHAIR")['test']

Loading Dataset



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

test-00000-of-00002.parquet:   0%|          | 0.00/375M [00:00<?, ?B/s]

test-00001-of-00002.parquet:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4863 [00:00<?, ? examples/s]

In [6]:
generations_file_path = '/content/drive/My Drive/CS 5787/Project/OpenCHAIR-Adjectives/example_gen_file.csv'
df = pd.read_csv(generations_file_path)

In [7]:
df['gt_caption'] = och_dataset['text'][:len(df)]

In [8]:
import spacy
from tqdm.auto import tqdm
spacy.require_gpu()

def is_concrete(noun, concretness, t=2.5):
    if noun in concretness:
        return concretness[noun] > t
    return False

def extract_adjs(captions, conc_df):
    nlp = spacy.load("en_core_web_sm")
    adjs = []
    for caption in tqdm(captions):
        doc = nlp(caption.lower())
        cur_adjs = [token.lemma_ for token in doc if token.pos_ == 'ADJ' and is_concrete(token.lemma_, conc_df)]
        adjs.append(cur_adjs)
    return adjs

In [9]:
df['generated_adjs'] = extract_adjs(df.generated_caption.tolist(), word_conc)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


  0%|          | 0/2000 [00:00<?, ?it/s]

In [10]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install torch --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.1.1
    Uninstalling accelerate-1.1.1:
      Successfully uninstalled accelerate-1.1.1


In [11]:
import bitsandbytes as bnb
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
)

def load_llm_pipe():
    llm_ckpt = "meta-llama/Llama-2-7b-chat-hf"
    tokenizer = AutoTokenizer.from_pretrained(llm_ckpt)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.pad_token = "[PAD]"
    tokenizer.padding_side = "left"

    bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                    bnb_4bit_quant_type="nf4",
                                    bnb_4bit_compute_dtype=torch.float16,
                                    bnb_4bit_use_double_quant=True)

    model = AutoModelForCausalLM.from_pretrained(llm_ckpt,
                                                 quantization_config=bnb_config,device_map="auto",
                                                 cache_dir=None)
    pipe = pipeline("text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    trust_remote_code=True,
                    device_map="auto",
                    batch_size=32)
    return pipe

print("\nLoading LLM\n")
llm_pipe = load_llm_pipe()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]


Loading LLM



tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


In [12]:
from functools import lru_cache
from torch.utils.data import Dataset

class ListDataset(Dataset):
     def __init__(self, original_list):
        self.original_list = original_list
     def __len__(self):
        return len(self.original_list)

     def __getitem__(self, i):
        return self.original_list[i]

def parse_ans(ans):
    ans_word_list = ans.lower().replace(',','').replace('.','').replace(';','').replace('\n',' ').split(' ')
    if 'yes' in ans_word_list:
        return 'yes'
    elif 'no' in ans_word_list or 'not' in ans_word_list:
        return 'no'
    elif 'unsure' in ans_word_list:
        return 'unsure'
    else:
        return 'ERROR: '+';'.join(ans_word_list)

def make_prompt(cap, adj, tokenizer):
    _prompt = f'''Here are a few descriptions of an image: {cap}\nDoes the image contain the following adjective: {adj}?\nAnswer yes/no/unsure.\n The answer is: '''
    prompt = tokenizer.apply_chat_template([{'role':'user', "content":_prompt}], tokenize=False)
    return prompt

@lru_cache(maxsize=None)
def get_answers(caps_flat, adjs_flat, pipe):
    prompts = [make_prompt(cap, adj, pipe.tokenizer) for cap,adj in zip(caps_flat, adjs_flat)]
    dataset = ListDataset(prompts)

    outputs = []
    with tqdm(total=len(prompts)) as pbar:
        for out in pipe(dataset, max_new_tokens=8, do_sample=False, num_return_sequences=1):
            outputs.append(out)
            pbar.update(1)

    outputs = [outputs[i][0]['generated_text'][len(prompts[i]):].strip() for i in range(len(outputs))]
    outputs = [parse_ans(out) for out in outputs]
    return outputs

def flatten_data(df):
    caps_flat, adjs_flat = [], []
    for cap, adjs in zip(df.gt_caption, df.generated_adjs):
        for adj in adjs:
            caps_flat.append(cap)
            adjs_flat.append(adj)
    return tuple(caps_flat), tuple(adjs_flat)

def unflatten_responses(responses_flat, df):
    responses_unflat = []
    i=0
    for adjs in df.generated_adjs:
        cur_responses = []
        for adj in adjs:
            cur_responses.append(responses_flat[i])
            i+=1
        responses_unflat.append(cur_responses)

    assert(len(responses_unflat) == len(df.generated_adjs))
    return responses_unflat

def apply_ignore_words(responses_flat, adjs_flat):
    ignore_words = ['painting', 'drawing', 'photo', 'picture', 'portrait', 'photograph']
    for i, adj in enumerate(adjs_flat):
        if adj in ignore_words:
            responses_flat[i] = 'ignore'
    return responses_flat

def get_llm_responses(df, llm_pipe):

    caps_flat, adjs_flat = flatten_data(df)
    responses_flat = get_answers(caps_flat, adjs_flat, llm_pipe)
    responses_flat = apply_ignore_words(responses_flat, adjs_flat)
    responses = unflatten_responses(responses_flat,df)

    return responses

print("\nGetting LLM Responses\n")
llm_responses = get_llm_responses(df, llm_pipe)


Getting LLM Responses



  0%|          | 0/571 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [13]:
print(llm_responses)

[[], [], [], ['no'], [], ['no'], [], [], [], [], [], [], [], [], [], [], ['no', 'no'], [], [], [], ['no'], [], ['no'], [], [], [], [], [], [], ['no', 'no', 'yes'], [], ['no'], [], ['no'], [], [], ['no', 'no', 'no'], ['no'], [], [], [], ['no'], ['no'], ['no'], [], [], ['no'], ['no', 'no'], ['no'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['no'], ['no'], [], ['no'], [], [], [], [], [], [], [], ['no', 'no', 'no'], [], ['no'], [], ['no'], [], [], [], [], ['no'], [], [], ['no'], [], [], [], [], [], [], ['no'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['no', 'no'], [], [], [], [], [], [], ['no'], [], [], [], [], [], ['no'], ['no'], [], ['no'], ['no', 'no'], ['no', 'no'], ['no'], ['unsure', 'unsure'], [], [], ['no'], [], ['yes'], [], [], [], [], [], ['no'], [], [], [], [], ['no'], [], [], [], ['no', 'no'], [], [], [], [], ['no', 'no'], [], [], [], [], [], [], [], [], [], [], [], [], ['no', 'yes'], [], ['no'], ['no', 'no'], [], [], [], [], [], 

In [14]:
def get_och_score(llm_responses):
    responses = []
    [responses.extend(resp_per_cap) for resp_per_cap in llm_responses]
    data = pd.Series(responses).str.lower().str.strip()
    dv = data.value_counts()
    d = dv.to_dict()
    return d['no'] / (d['yes'] + d['no'])

OpenCHAIR_score = get_och_score(llm_responses)
print("\nOpenCHAIR Score: \n")
print(OpenCHAIR_score)


OpenCHAIR Score: 

0.9247706422018349
